In [4]:
from math import sqrt
from statsmodels.stats.proportion import proportion_confint
from numpy.random import seed
from numpy.random import rand
from numpy.random import randint
from numpy import mean 
from numpy import median
from numpy import percentile

# Introducción

La gran mayoría del aprendizaje automático se centra en estimar el rendimiento de un algoritmo con datos no vistos. Los intervalos de confianza son una forma de cuantificar la incertidumbre de una estimación.

# ¿Qué es un intervalo de confianza?

Un intervalo de confianza es un límite en la estimación de una variable en una población. Es un intervalo estadístico usado para cuantificar la incertidumbre en una estimación. Un intervalo de confianza es diferente de un intervalo de tolerancia que describe los límites de la muestra de los datos de la distribución. Es también diferente de un intervalo de predicción que describe los límites de una simple observación. El intervalo de confianza proporciona límites en un parámetro de la población, tales como la media, desviación estándar, etc. En el aprendizaje automático aplicado, podemos querer usar intervalos de confianza en la presentación de las abilidades de nuestro algoritmo predictivo. Por ejemplo, un intervalo de confianza podría ser usada para presentar las abilidades de un modelo de clasificación:

* Dada una muestra, existe una probabilidad del 95% de que el rango [x,y] cubra la verdadera precisión del modelo.

Los intervalos de confianza puede ser usados en la presentación del error en un modelo predictivo de regresión:

* Hay un 95% de probabilidad de que el rango [x,y] cubra el error verdadero del modelo.

La elección del 95% de confianza es muy común en la presentación de intervalos de confianza, también pueden ser usados otros valores.

El valor de un intervalo de confianza es la abilidad de cuantificar la incertidumbre de lo estimado. Nos proporciona un valor superior e inferior con una determinada probabilidad.

* **Pequeños Intervalos de Confianza:** estimación más precisa.

* **Grandes Intervalos de Confianza:** estimación menos precisa.

Un intervalo de confianza pequeño significa un pequeño margen de eror y que hemos sido relativamente precisos estimando. Un intervalo de confianza grande significa un margen de error elevado y una menor precisión a la hora de estimar. EL uso de los intervalos de confianza nos ayudará de dos maneras. En primer lugar, los intervalos le dan a los que usan modelos una compresión de como de bueno puede llegar a ser el modelo. EL segundo beneficio de los intervalos de confianza es la facilidad de comparar modelos. Si los intervalos de confianza de dos modelos están superpuestos, esto es una indicación de la equivalencia entre dos modelos, por lo que sería razonable seleccionar el modelo más interpretable.

# Intervalo para la precisión en clasificación

En los problemas de clasificación es muy común de hace uso de la métrica accuracy o el error de clasificación (inversa del accuracy) que describe la abilidad de clasificar que tiene un modelo predictivo. Lo que nos indica es la proporción de predicciones correctas o incorrectas realizadas por el modelo. Cada predicción es una decisión binaria que puede ser correcta o incorrecta. Técnicamente, es llamada experimento de Bernoulli. Las proporciones en un experimento de Bernoulli tienen una distribución específica llamada distribución Binomial. Afortunadamente, con una muesra de gran tamaño, podemos aproximar la distribución con una Gaussianna. En estadística, una sucesión de eventos independientes que pueden tomar el valor de éxito o fracaso es llamada proceso de Bernoulli. Para un N grande, la distribución de esta variable aleatoria se acerca a la distribución normal. Podemos hacer uso del supuesto de una distribución Gaussianna para calcular el intervalo de confianza:

$$interval = z \sqrt{\frac{error(1-error)}{n}}$$
$$interval = z \sqrt{\frac{accuracy(1-accuracy)}{n}}$$

Donde el **interval** es el radio del intervalo de confianza, error y accuracy se corresponden a las métricas de nuestro algoritmo, **n** es el tamaño de la muestra, y **z** es un valor crírico de la distribución Gaussianna. Comunmente usa valores críticos de la distribución Gaussianna y sus correspondientes niveles de significancia son los siguientes:

* 1.64(98%)

* 1.96(95%)

* 2.33(98%)

* 2.58(99%)

Por ejemplo, si consideramos un modelo con un error del 20%, en un conjunto de validación con 50 muestras. Podemos calcular el intervalo de confianza del 95% como

In [2]:
interval_error = 1.96*sqrt(0.2*(1-0.2)/50)
print('%.3f' % interval_error)

0.111


Tras esto podemos realizar las siguientes afirmaciones:

* El error de clasificación de nuestro modelo es de 20% +/- 1%

* Con un probabilidad del 95% el verdadero error de nuestro modelo se encuentra entre el 9% y el 31%

Debemos de recordar que un intervalo de confianza es una probabilidad dentro de un rango.

La función **proportion_confint()** implementa el intervalo de confianza como una proporción binomial. Por defecto toma una asunción Gaussianna para la distribución binomial, además de otras variaciones más sofisticadas son soportadas en el cálculo. La función toma como argumento el número de aciertos (o errores), el número de muestras, y el nivel de significancia. Por ejemplo supongamos el caso en el cuál un algoritmo ha acertado un total de 88 casos para una muestra de observaciones y deseamos obtener el intervalo de confianza del 95%.

In [3]:
#Vemos los límites superioes e inferiores de nuestro intervalo de confianza
lower, upper = proportion_confint(88, 100, 0.05)
print('lower=%.3f, upper=%.3f' % (lower, upper))

lower=0.816, upper=0.944


Podemos decir que con una probabilidad del 95%, nuestro algoritmo tendrá un accuracy entre el 81.6% y el 94.4%.

# Intervalos de Confianza no paramétricos

A menudo no conocemos la distribución para una medida de rendimiento elegida. Por lo tanto, no podemos realizar el cálculo del intervalo de confianza de una forma analítica.

Las asunciones que hay bajo los intervalos de confianza paramétricos son a menudo violados. La variable predictora alguna veces no sigue una distribución normal, incluso cuando es normal, la varianza de la distribución normal podría no ser igual para todos los niveles de la variable predictora.

En estos casos, el método de remuestreo bootstrap puede ser usado como un método no paramétrico para calcular los intervalos de confianza. Boostrap es un método de simulación de Monte Carlo donde las muestras son extraídas de un conjunto de datos fijos y finito con reemplazamiento y un parámetro es estimado en cada muestra. Este procedimiento lleva a una etimación robusta del verdadero parámetro de la población mediante el muestreo.

Este procedimiento puede ser usado para estimar las abilidades de un modelo predictivo fijando el modelo con cada muestra y evaluándolo con aquellas muestras que no están incluidas. La media o la mediana puede ser métricas que representen la abilidad de nuestro modelo cuando es evaluado con datos no vistos. Los intervalos de confianza pueden ser agregados seleccionando observaciones de la muestra de puntuaciones y calculando percentiles específicos.

Recordar que un percentil es un valor extraido de la muestra ordenada donde se cumple que un porcentaje de la muestra cae por debajo de ese valor. Por ejemplo, el percentil 70 de una muestra indica que el 70% de las muestras caen por debajo de ese valor. El percentil 50 es la mediana. Antes de seleccionar un percentil debemos de seleccionar un nivel de significancia para el nivel de confianza, por ejemplo si seleccionamos un intervalo de confianza del 95%, representa un nivel de significancia del 5%. Puesto que el intervalo de confianza es simétrico respecto a la mediana, debemos seleccionar el percentil 2.5th y el percentil 97.5th para obtener el rango completo.

In [11]:
#fijamos la semilla
seed(1)

#generamos un conjunto de datos
dataset = 0.5 + rand(1000)*0.5

#hacemos uso de la técnica bootstrap
scores = list()
for _ in range(100):
    indices = randint(0,1000,1000)
    sample = dataset[indices]
    score = mean(sample)
    scores.append(score)

print('El percentil 50th es: %.3f' % median(scores))

#Calculamos los límites del intervalo de confianza 95%
alpha = 5
lower_p = alpha/2.0
lower = max(0.0, percentile(scores, lower_p))

upper_p = (100-alpha) + alpha/2.0
upper = min(1.0, percentile(scores, upper_p))

print('El percentil %.1fth es: %.3f' % (lower_p, lower))
print('El percentil %.1fth es: %.3f' % (upper_p, upper))

El percentil 50th es: 0.750
El percentil 2.5th es: 0.741
El percentil 97.5th es: 0.757


Esto nos indica que con una probabilidad del 95% el valor verdadero de la media de nuestro conjunto de datos se encuentra entre los valores 0.741 y 0.757.